# Deploying a Hugging Face (DistilBERT Sentiment Classification) model on Verta

Within Verta, a "Model" can be any arbitrary function: a traditional ML model (e.g., sklearn, PyTorch, TF, etc); a function (e.g., squaring a number, making a DB function etc.); or a mixture of the above (e.g., pre-processing code, a DB call, and then a model application.) See more [here](https://docs.verta.ai/verta/registry/concepts).

This notebook provides an example of how to deploy a Hugging Face model on Verta as a Verta Standard Model by extending [VertaModelBase](https://verta.readthedocs.io/en/master/_autogen/verta.registry.VertaModelBase.html?highlight=VertaModelBase#verta.registry.VertaModelBase).

Updated for Verta version: 0.21.0

This notebook walks through creating a few DistilBERT and BERT sentiment classification models, logging them to the Verta platform, and deploying them.

<a href="https://colab.research.google.com/github/VertaAI/examples/blob/main/deployment/huggingface/distilbert-sentiment-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Imports

In [1]:
!python -m pip install verta
!python -m pip install transformers

In [2]:
from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)

### 0.1 Verta import and setup

In [3]:
# import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST'] = 

In [4]:
import os
from verta import Client
from verta.utils import ModelAPI

client = Client(os.environ['VERTA_HOST'])

---

## 1. Register model

### 1.1 Define model

A model has to exist before we can register and deploy a version, so we will instantiate one here in our notebook.

This model class will be an extensible wrapper over a [pre-trained Hugging Face classifier](https://huggingface.co/transformers/index.html).

In [5]:
from verta.registry import VertaModelBase, verify_io

class _ModelBase(VertaModelBase):
    MODEL = None

    def __init__(self, artifacts=None):
        self.model = pipeline(
            task="sentiment-analysis",
            model=AutoModelForSequenceClassification.from_pretrained(self.MODEL),
            tokenizer=AutoTokenizer.from_pretrained(self.MODEL),
        )

    @verify_io
    def predict(self, texts):
        return self.model(texts)

    def example(self):
        return [
            "I like you",
            "I don't like this film",
        ]


class DistilBERT(_ModelBase):
    MODEL = "distilbert-base-uncased-finetuned-sst-2-english"

    def predict(self, texts):
        sentiments = super(DistilBERT, self).predict(texts)

        return sentiments

As a sanity check, we can validate that our model is instantiable and can produce predictions.

In [6]:
distilbert = DistilBERT()

texts = distilbert.example()
prediction = distilbert.predict(texts)

print(texts)
print(prediction)

### 1.2 Register model for deployment

Now that the model is in a good shape, we can register it into the Verta platform.

We'll create a [registered model](https://verta.readthedocs.io/en/master/_autogen/verta.registry.entities.RegisteredModel.html) for our DistilBERT text classification models  
and a [version](https://verta.readthedocs.io/en/master/_autogen/verta.registry.entities.RegisteredModelVersion.html) to associate this particular model with.

All of these can be viewed in the Verta web app once they are created.

In [7]:
registered_model = client.get_or_create_registered_model(
    "DistilBERT",
    desc="Models trained for textual sentiment classification.",
    labels=["NLP", "Classification", "Neural Net"],
)

In [8]:
from verta.environment import Python

distilbert_model = registered_model.create_standard_model(
    model_cls=DistilBERT,
    model_api=ModelAPI(texts, prediction),
    environment=Python([
        "dill",  # used by torch internally for serialization
        "torch",
        "transformers",
    ]),
    # from https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english
    attrs={
        "learning_rate": 1e-5,
        "batch_size": 32,
        "max_seq_length": 128,
        "num_train_epochs": 3,
        "accuracy": .913,
    },
)

### 1.3 Deploy model to endpoint

In [9]:
from verta.endpoint.resources import Resources

resources = Resources(memory="1Gi")
transformers_cache_env = {"TRANSFORMERS_CACHE": "/tmp"}
 
sentiment_endpoint = client.get_or_create_endpoint("classify-sentiment")
sentiment_endpoint.update(  # can take ~10min for this model
    distilbert_model,
    resources=resources,
    env_vars=transformers_cache_env,
    wait=True,
)

In [10]:
deployed_model = sentiment_endpoint.get_deployed_model()

print(texts)
print(deployed_model.predict(texts))

---

## 2. Register additional models

One key aspect of the Verta platform is being able to organize your various models in a easy-to-navigate structure.  
We can register additional BERT models—and be able to reference and deploy them as well.

### 2.1 Define models

In [11]:
registered_model = client.get_or_create_registered_model(
    "BERT",
    labels=["NLP", "Classification", "Neural Net"],
)

In [12]:
class BERT(_ModelBase):
    MODEL = "textattack/bert-base-uncased-imdb"

    @verify_io
    def predict(self, texts):
        sentiments = super(BERT, self).predict(texts)

        # fix labels
        for sentiment in sentiments:
            if sentiment['label'] == "LABEL_0":
                sentiment['label'] = "NEGATIVE"
            else:  # "LABEL_1"
                sentiment['label'] = "POSITIVE"

        return sentiments
    
bert_model = registered_model.create_standard_model(
    model_cls=BERT,
    model_api=ModelAPI(texts, prediction),
    environment=Python(["dill", "torch", "transformers"]),
    name="English BERT",
    labels=["BERT", "English"],
    # from https://huggingface.co/textattack/bert-base-uncased-imdb
    attrs={
        "learning_rate": 2e-5,
        "batch_size": 16,
        "max_seq_length": 128,
        "num_train_epochs": 5,
        "accuracy": .89088,
    },
)

In [13]:
class GermanBERT(_ModelBase):
    MODEL = "oliverguhr/german-sentiment-bert"

    @verify_io
    def predict(self, texts):
        sentiments = super(GermanBERT, self).predict(texts)
        
        # fix labels
        for sentiment in sentiments:
            sentiment['label'] = sentiment['label'].upper()

        return sentiments

german_model = registered_model.create_standard_model(
    model_cls=GermanBERT,
    model_api=ModelAPI(texts, prediction),
    environment=Python(["dill", "torch", "transformers"]),
    name="German",
    labels=["BERT", "German"],
    # from http://www.lrec-conf.org/proceedings/lrec2020/pdf/2020.lrec-1.202.pdf
    attrs={
        "learning_rate": 2e-5,
        "batch_size": 32,
        "max_seq_length": 256,
        "num_train_epochs": 3,
        "f1": .9639
    }
)

In [14]:
class MultilingualBERT(_ModelBase):
    MODEL = "nlptown/bert-base-multilingual-uncased-sentiment"

    def __init__(self, artifacts=None):
        super(MultilingualBERT, self).__init__()
        self.model._postprocess_params.update({"top_k": None})  # this model has 5 categories, and we'll need to make it 2

    @verify_io
    def predict(self, texts):
        texts_scores = super(MultilingualBERT, self).predict(texts)

        # fix labels and scores
        sentiments = []
        for scores in texts_scores:
            # aggregate negative and positive scores
            negative_scores = filter(lambda score: score['label'] in {"1 star", "2 stars", "3 stars"}, scores)
            positive_scores = filter(lambda score: score['label'] in {"4 stars", "5 stars"}, scores)
            negative_score = sum(score['score'] for score in negative_scores)
            positive_score = sum(score['score'] for score in positive_scores)

            # select greater value as sentiment
            if positive_score > negative_score:
                label, score = "POSITIVE", positive_score
            else:
                label, score = "NEGATIVE", negative_score
            sentiments.append({'label': label, 'score': score})

        return sentiments

multilingual_model = registered_model.create_standard_model(
    model_cls=MultilingualBERT,
    model_api=ModelAPI(texts, prediction),
    environment=Python(["dill", "torch", "transformers"]),
    name="Multilingual",
    labels=["BERT", "English", "German"],
    # from https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment
    attrs={  # example values; true hyperparameters not provided by model contributors
        "learning_rate": 2e-5,
        "batch_size": 16,
        "max_seq_length": 128,
        "num_train_epochs": 3,
        "accuracy": .95,
    },
)

### 2.2 Deploy models to endpoints

In [15]:
sentiment_endpoint.update(
    bert_model,
    resources=resources,
    env_vars=transformers_cache_env,
    wait=True,
)
deployed_model = sentiment_endpoint.get_deployed_model()
deployed_model.predict(texts)

In [16]:
sentiment_endpoint.update(
    german_model,
    resources=resources,
    env_vars=transformers_cache_env,
    wait=True,
)
deployed_model = sentiment_endpoint.get_deployed_model()
deployed_model.predict(["Das ist gut", "Allergien machen keinen Spaß"])

In [17]:
sentiment_endpoint.update(
    multilingual_model,
    resources=Resources(memory="2Gi"),
    env_vars=transformers_cache_env,
    wait=True,
)
deployed_model = sentiment_endpoint.get_deployed_model()
deployed_model.predict(["Cette pâte est agréablement feuilletée"])

---